# Imports
Import python libraries

In [25]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from itertools import combinations, product

# Set dataset parameters
Choose the attributes to load the dataset.

In [26]:


detector = 'mtcnn_serfiq'  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_model = 'QMagFace'  # 'QMagFace', 'QMagFace_SR',"ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'

#quality_models = ['ser_fiq'] #  'ser_fiq','tface'
#embedding_models = ['Facenet512','QMagFace','ArcFace'] 
#quality_models = ['ser_fiq','tface']

# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [27]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'

#calibration_db0 = ['XQLFW','SCFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',
#image_filters0 = ['yaw', 'gender', 'race','emotion']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'

calibration_db = ['XQLFW'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW','UTKface','ENFSI',
#'XQLFW','UTKface',"SCFace",'ForenFace',



In [28]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Input and output directories
Where the sql database is and which folder the results are saved.

In [29]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset_21b'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'experiment')

# Preprocess data
Connect to sql database.

In [30]:
from sql_face.alchemy import get_session
from sql_face.tables import *

In [31]:
session = get_session(input_dir, db_name)

In [32]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [33]:
results1 = session.query(Image,FaceImage) \
.select_from(Image).filter(Image.source == 'XQLFW') \
.join(CroppedImage).join(Detector).filter(Detector.name == detector) \
.join(FaceImage).join(EmbeddingModel).filter(EmbeddingModel.name == embedding_model).all()



In [34]:
session.close()

Creo el dataframe con los resultados

In [35]:
im_cols = ['identity',  'path' , 'image_id']
fi_cols = ['embeddings']
#fixed_cols = ['Detector','E_Model','Q_Model','Q_group','Quality']

df1 = pd.DataFrame([ [getattr(r[0],attr)   for attr in im_cols]  + [r[1].embeddings]
                   for r in results1],
                   columns=im_cols + fi_cols )

In [36]:
identities = set(df1.identity)

Defino la funcion de distancia o similitud

In [37]:
#hago la funcion coseno por hacer una
def similitud(a,b):
    a1 = np.array(a)
    b1 = np.array(b)

    return( np.dot(a1,b1)/(np.linalg.norm(a1)*np.linalg.norm(b1)))


Buscar dentro de la misma identidad

In [38]:
df_resultado = pd.DataFrame(columns=['path1', 'path2', 'resultado'])

In [ ]:
#identities = set(df1.identity)
identities = []

In [43]:

for identity in tqdm(identities) :
    dfx = df1.loc[(df1.identity == identity)]
    if len(dfx) > 1:
        combinaciones = list(combinations(dfx.iterrows(), 2))

        # Iterar sobre las combinaciones
        for comb in tqdm(combinaciones):
            index1, row1 = comb[0]
            index2, row2 = comb[1]
    
            path1 = row1['path']
            path2 = row2['path']
    
            # Realizar la operación deseada (ejemplo: suma de los embeddings)
            resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
            df_resultado = df_resultado.append({
            'path1': path1,
            'path2': path2,
            'resultado': resultado
            }, ignore_index=True)
        
        


0it [00:00, ?it/s]


In [41]:
if len(resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'misma_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'misma_identidad.xlsx'))

In [ ]:
identities = set(df1.identity)
#identities = []

In [ ]:
for identity in identities :
    dfx = df1.loc[(df1.identity == identity)]
    dfy = df1.loc[(df1.identity > identity)]
    
    combinaciones = list(combinations(dfx.iterrows(), dfy.iterrows()))

        # Iterar sobre las combinaciones
    for comb in tqdm(combinaciones):
        index1, row1 = comb[0]
        index2, row2 = comb[1]
    
        path1 = row1['path']
        path2 = row2['path']
    
        # Realizar la operación deseada (ejemplo: suma de los embeddings)
        resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
        df_resultado = df_resultado.append({
        'path1': path1,
        'path2': path2,
        'resultado': resultado
        }, ignore_index=True)
        

In [ ]:
if len(resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'diferente_identidad.xlsx'))

In [42]:
len(identities)

5715